<a href="https://colab.research.google.com/github/asosawelford/Urban-Sound-Classification-with-PyTorch/blob/main/MNIST_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Training a Neural Network with PyTorch

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001


class FeedForwardNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        #Sequential contains different layers
        #images in MNIST ar 28*28 pixels, we'll use 256 images
        # 10 classes in mnist dataset
        self.dense_layers = nn.Sequential(
            nn.Linear(28 * 28, 256), 
            nn.ReLU(),
            nn.Linear(256, 10)
        )
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        x = self.flatten(input_data)
        logits = self.dense_layers(x)
        predictions = self.softmax(logits)
        return predictions

##################################### 
#Download dataset
#####################################

def download_mnist_datasets():
    train_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )
    validation_data = datasets.MNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    return train_data, validation_data


def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader


def train_single_epoch(model, data_loader, loss_fn, optimiser, device):
    for input, target in data_loader:
        input, target = input.to(device), target.to(device)

        # calculate loss
        prediction = model(input)
        loss = loss_fn(prediction, target)

        # backpropagate error and update weights
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    print(f"loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimiser, device, epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_single_epoch(model, data_loader, loss_fn, optimiser, device)
        print("---------------------------")
    print("Finished training")


if __name__ == "__main__":

    # download data and create data loader
    train_data, _ = download_mnist_datasets()
    train_dataloader = create_data_loader(train_data, BATCH_SIZE)

    # construct model and assign it to device
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using {device}")
    feed_forward_net = FeedForwardNet().to(device)
    print(feed_forward_net)

    # initialise loss funtion + optimiser
    loss_fn = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(feed_forward_net.parameters(),
                                 lr=LEARNING_RATE)

    # train model
    train(feed_forward_net, train_dataloader, loss_fn, optimiser, device, EPOCHS)

    # save model
    torch.save(feed_forward_net.state_dict(), "feedforwardnet.pth")
    print("Trained feed forward net saved at feedforwardnet.pth")

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

Using cuda
FeedForwardNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense_layers): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)
Epoch 1
loss: 1.5089529752731323
---------------------------
Epoch 2
loss: 1.498639702796936
---------------------------
Epoch 3
loss: 1.4959641695022583
---------------------------
Epoch 4
loss: 1.4853121042251587
---------------------------
Epoch 5
loss: 1.4802559614181519
---------------------------
Epoch 6
loss: 1.4741896390914917
---------------------------
Epoch 7
loss: 1.4731254577636719
---------------------------
Epoch 8
loss: 1.4742189645767212
---------------------------
Epoch 9
loss: 1.4737029075622559
---------------------------
Epoch 10
loss: 1.4729796648025513
---------------------------
Finished training
Trained feed forward net 

# Making Predictions with PyTorch
Loading the model and making inferences


In [ ]:
class_mapping = [
                 "0",
                 "1",
                 "2",
                 "3",
                 "4",
                 "5",
                 "6",
                 "7",
                 "8",
                 "9"
]

def predict(model, input, target, class_mapping):
  model.eval()
  with torch.no_grad():
    predictions = model(input)
    # Tensor object, 2 dimentional (1,10) 1 = num of samples passed as input, 10 = number of classes our model tries to predict
    predicted_index = predictions[0].argmax(0)
    predicted = class_mapping[predicted_index]
    expected = class_mapping[target]
    return predicted, expected

# Load the model
feed_foward_net = FeedForwardNet()
state_dict = torch.load("feedforwardnet.pth")
feed_foward_net.load_state_dict(state_dict)

# Load validation dataset
_, validation_data = download_mnist_datasets()

# Get a sample from the validation set for inference
for i in range(25):
  input, target = validation_data[i][0], validation_data[i][1] #get target for initial sample

# Make an inference (new function "predict" it returns both what the model predicted and what we expect given the target)

  predicted, expected = predict(feed_foward_net, input, target, class_mapping)

  print(f"Predicted ='{predicted}', expected = '{expected}'")


Predicted ='7', expected = '7'
Predicted ='2', expected = '2'
Predicted ='1', expected = '1'
Predicted ='0', expected = '0'
Predicted ='4', expected = '4'
Predicted ='1', expected = '1'
Predicted ='4', expected = '4'
Predicted ='9', expected = '9'
Predicted ='5', expected = '5'
Predicted ='9', expected = '9'
Predicted ='0', expected = '0'
Predicted ='6', expected = '6'
Predicted ='9', expected = '9'
Predicted ='0', expected = '0'
Predicted ='1', expected = '1'
Predicted ='5', expected = '5'
Predicted ='9', expected = '9'
Predicted ='7', expected = '7'
Predicted ='3', expected = '3'
Predicted ='4', expected = '4'
Predicted ='9', expected = '9'
Predicted ='6', expected = '6'
Predicted ='6', expected = '6'
Predicted ='5', expected = '5'
Predicted ='4', expected = '4'
